<a href="https://colab.research.google.com/github/cacos8/colab/blob/main/Subtitutlador_SIN_OPCIONES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar video y transcribir

In [ ]:
from google.colab import files
uploaded = files.upload()

video_file = next(iter(uploaded))
print("Vídeo cargado:", video_file)


Saving WhatsApp Video 2025-06-06 at 08.36.46.mp4 to WhatsApp Video 2025-06-06 at 08.36.46.mp4
Vídeo cargado: WhatsApp Video 2025-06-06 at 08.36.46.mp4


In [ ]:
!pip install -q openai-whisper

import whisper

# Cargar modelo
model = whisper.load_model("small")  # Puedes usar "tiny" si quieres más velocidad

# Transcripción en español
result = model.transcribe(video_file, language="es", verbose=True)

# Guardar las frases segmentadas
segments = result["segments"]

# Mostrar un resumen
for seg in segments[:5]:
    print(f"[{seg['start']:.2f}s - {seg['end']:.2f}s] {seg['text']}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.5 MB/

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 75.8MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:08.000]  El perro de Rock no tiene rabo porque Ramón Ramírez se lo ha cortado.
[0.00s - 8.00s]  El perro de Rock no tiene rabo porque Ramón Ramírez se lo ha cortado.


# Revisión ortográfica

In [ ]:
# Celda X: Revisión ortográfica (versión parcheada) de los segmentos

# 1) Instalar la librería de corrección ortográfica
!pip install -q pyspellchecker

from spellchecker import SpellChecker
from collections import defaultdict

# 2) Crear el SpellChecker para español
spell = SpellChecker(language='es')

# 3) Buscar palabras mal escritas
errores = []  # lista de tuplas (índice_segmento, palabra_errónea, lista_sugerencias)

for idx, seg in enumerate(segments, start=1):
    texto = seg['text']
    palabras = [w.strip("¡!¿?.,;:()[]\"'") for w in texto.split()]
    mal = spell.unknown(palabras)
    for palabra in mal:
        # Intentar obtener sugerencias; si devuelve None, usar lista vacía
        cands = spell.candidates(palabra)
        sugerencias = list(cands) if cands else []
        errores.append((idx, palabra, sugerencias))

# 4) Mostrar resultados agrupados
if not errores:
    print("✅ No se han encontrado palabras potencialmente mal escritas.")
else:
    print("⚠️ Se han detectado posibles errores ortográficos:\n")
    agrupado = defaultdict(list)
    for idx, palabra, sugerencias in errores:
        agrupado[idx].append((palabra, sugerencias))

    for idx in sorted(agrupado):
        print(f"Segmento {idx}: “{segments[idx-1]['text']}”")
        for palabra, sugerencias in agrupado[idx]:
            if sugerencias:
                print(f"   – {palabra!r} → sugerencias: {', '.join(sugerencias)}")
            else:
                print(f"   – {palabra!r} → (sin sugerencias)")
        print()




⚠️ Se han detectado posibles errores ortográficos:

Segmento 1: “ El perro de Rock no tiene rabo porque Ramón Ramírez se lo ha cortado.”
   – 'ramírez' → (sin sugerencias)
   – 'rock' → sugerencias: roce, roca
   – 'tiene' → sugerencias: tizne



In [ ]:
# Celda X: Revisar y editar los segmentos que produjo Whisper

import ipywidgets as widgets
from IPython.display import display

# 1) Formatear los segmentos actuales en líneas "start|end|texto"
lines = []
for seg in segments:
    lines.append(f"{seg['start']:.3f}|{seg['end']:.3f}|{seg['text']}")

initial_text = "\n".join(lines)

# 2) Crear un Textarea para mostrar y editar esos segmentos
textarea = widgets.Textarea(
    value=initial_text,
    placeholder="Cada línea debe tener formato: start|end|texto",
    description="Editar segmentos:",
    layout=widgets.Layout(width="100%", height="400px")
)

# 3) Botón para guardar los cambios y reconstruir la lista 'segments'
save_button = widgets.Button(description="Guardar cambios", button_style="success")
output    = widgets.Output()

def on_save_clicked(b):
    with output:
        output.clear_output()
        edited_lines = textarea.value.strip().split("\n")
        new_segments = []
        for idx, line in enumerate(edited_lines, start=1):
            if not line.strip():
                continue
            parts = line.split("|", 2)
            if len(parts) != 3:
                print(f"⚠️ Línea {idx} mal formateada (esperado start|end|texto):\n    {line}")
                return
            try:
                start = float(parts[0])
                end   = float(parts[1])
            except ValueError:
                print(f"⚠️ Línea {idx}: tiempos inválidos:\n    {line}")
                return
            text = parts[2].strip()
            new_segments.append({"start": start, "end": end, "text": text})
        # Reemplazar la lista global 'segments' con los valores corregidos
        global segments
        segments = new_segments
        print(f"✅ Se han guardado {len(segments)} segmentos (listos para generar el SRT/ASS).")

save_button.on_click(on_save_clicked)

# 4) Mostrar el cuadro de texto y el botón
display(textarea, save_button, output)


Textarea(value='0.000|8.000| El perro de Rock no tiene rabo porque Ramón Ramírez se lo ha cortado.', descripti…

Button(button_style='success', description='Guardar cambios', style=ButtonStyle())

Output()

# Creador de subtítulos
Para cambiar la altura a la que salen hay que cambiar "margin_v". Igualmente con los colores y demás opciones que se cambian directamente en código.
* Alignment=2   → centrar horizontal y anclar ABAJO (encima de margin_v)
* MarginL/MarginR = margin_lr
* MarginV = margin_v
* PrimaryColour = blanco (&H00FFFFFF)
* SecondaryColour = verde puro (&H0000FF00)
* OutlineColour = negro (&H00000000) para contorno

In [ ]:
# Celda 3.2 (definitiva): Karaoke con wrapping manual, centrado ABAJO, y resaltado palabra a palabra
# (Solo se ha modificado el bloque de generación de eventos para pintar cada palabra en verde al sonar
#  y en blanco el resto, manteniendo intacto el centrado y márgenes).

import subprocess
from moviepy.editor import VideoFileClip
from google.colab import files

# 1) Cargar el vídeo para obtener su resolución
video = VideoFileClip(video_file)
W, H = video.w, video.h

# 2) Calcular dinámicamente fontsize y márgenes:
#    - fontsize  = 4% de la altura (mínimo 16 px)
#    - margin_v   = 5% de la altura  (zona segura inferior)
#    - margin_lr  = 5% del ancho    (zona segura lateral para wrapping)
fontsize  = max(16, int(H * 0.04))
margin_v   = int(H * 0.15)
margin_lr  = int(W * 0.05)

# 3) Calcular el número máximo aproximado de caracteres por línea
#    Usamos un factor aproximado de 0.6 * fontsize para ancho de carácter
max_chars = max(20, int((W - 2 * margin_lr) / (fontsize * 0.6)))

# 4) Construir la cabecera ASS con estilo ajustado:
#    * Alignment=2   → centrar horizontal y anclar ABAJO (encima de margin_v)
#    * MarginL/MarginR = margin_lr
#    * MarginV = margin_v
#    * PrimaryColour = blanco (&H00FFFFFF)
#    * SecondaryColour = verde puro (&H0000FF00) para karaoke
#    * OutlineColour = negro (&H00000000) para contorno
# tipo de letra: Style: Karaoke,DejaVu Sans,{fontsize},&H00FFFFFF,&H0000FF00,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,0,2,{margin_lr},{margin_lr},{margin_v},1
ass_header = f"""[Script Info]
ScriptType: v4.00+
PlayResX: {W}
PlayResY: {H}

[V4+ Styles]
Format: Name,Fontname,Fontsize,PrimaryColour,SecondaryColour,OutlineColour,BackColour,Bold,Italic,Underline,StrikeOut,ScaleX,ScaleY,Spacing,Angle,BorderStyle,Outline,Shadow,Alignment,MarginL,MarginR,MarginV,Encoding
;  PrimaryColour   = blanco         (&H00FFFFFF)
;  SecondaryColour = verde puro     (&H0000FF00)
;  OutlineColour   = negro          (&H00000000)

Style: Karaoke,Arial,{fontsize},&H00FFFFFF,&H00006EED,&H00000000,&H00000000,1,0,0,0,100,100,0,0,1,2,0,2,{margin_lr},{margin_lr},{margin_v},1


[Events]
Format: Layer,Start,End,Style,Name,MarginL,MarginR,MarginV,Effect,Text
"""

lines = [ass_header]

def to_ass_time(t: float) -> str:
    """
    Convierte segundos (float) a formato h:mm:ss.cs (centésimas).
    Ejemplo: 65.34 → "0:01:05.34".
    """
    h = int(t // 3600)
    m = int((t % 3600) // 60)
    s_full = t % 60
    s = int(s_full)
    cs = int(round((s_full - s) * 100))
    return f"{h:d}:{m:02d}:{s:02d}.{cs:02d}"

# 5) Construir eventos ASS: un evento POR CADA palabra,
#    coloreando esa palabra en verde y el resto en blanco,
#    y manteniendo el wrapping en máximo dos líneas.
#    Asumimos que `segments` proviene de Whisper (Celda 2).
for seg in segments:
    start_seg = seg["start"]
    end_seg   = seg["end"]
    text      = seg["text"].strip()
    words     = text.split()
    if not words:
        continue

    total_dur    = end_seg - start_seg
    dur_per_word = total_dur / len(words)

    # Dividir las palabras en dos listas (líneas) sin exceder max_chars
    line1 = []
    current_len = 0
    for w in words:
        if current_len + len(w) <= max_chars or not line1:
            line1.append(w)
            current_len += len(w) + 1
        else:
            break
    idx_split = len(line1)
    line2 = words[idx_split:]

    # Para cada palabra i, creamos un evento ASS que pinta solo esa palabra en verde
    for i, w in enumerate(words):
        start_i = start_seg + i * dur_per_word
        end_i   = start_i + dur_per_word
        ass_start = to_ass_time(start_i)
        ass_end   = to_ass_time(end_i)

        # Construir texto coloreado para las dos líneas
        colored_lines = []

        # Línea 1
        if line1:
            parts1 = []
            for j, wtext in enumerate(line1):
                if j == i:  # si coincide índice con la palabra activa
                    parts1.append(f"{{\\c&H00006EED&}}{wtext}{{\\c&HFFFFFF&}}")
                else:
                    parts1.append(wtext)
            colored_lines.append(" ".join(parts1))

        # Línea 2
        if line2:
            parts2 = []
            for j, wtext in enumerate(line2, start=idx_split):
                if j == i:
                    parts2.append(f"{{\\c&H00006EED&}}{wtext}{{\\c&HFFFFFF&}}")
                else:
                    parts2.append(wtext)
            colored_lines.append(" ".join(parts2))

        # Unir ambas líneas con salto de línea ASS si hay dos líneas
        kara_text = r"\N".join(colored_lines) if len(colored_lines) == 2 else colored_lines[0]

        # Formato de Dialogue:
        # Layer,Start,End,Style,Name,MarginL,MarginR,MarginV,Effect,Text
        # El estilo ASS ya define Alignment y márgenes, por eso ponemos 0,0,0 en MarginL/MarginR/MarginV.
        lines.append(
            f"Dialogue: 0,{ass_start},{ass_end},Karaoke,,0,0,0,,{kara_text}"
        )

# 6) Escribir karaoke.ass en disco
karaoke_ass = "\n".join(lines)
with open("karaoke.ass", "w", encoding="utf-8") as f:
    f.write(karaoke_ass)

print(
    f"► 'karaoke.ass' creado: resolución {W}×{H}, fontsize={fontsize}, "
    f"margenes L/R={margin_lr}, V={margin_v}, max_chars={max_chars}"
)


► 'karaoke.ass' creado: resolución 848×480, fontsize=19, margenes L/R=42, V=288, max_chars=67


# Hacer el video

In [ ]:
# Celda 3.3: quemar (hardcode) el karaoke.ass en el vídeo con ffmpeg
!ffmpeg -y -i "{video_file}" -vf "ass=karaoke.ass" -c:a copy video_karaoke.mp4

# Mostrar y descargar el vídeo generado
from IPython.display import Video, HTML, display
from moviepy.editor import VideoFileClip

# Obtener dimensiones del vídeo original para el reproductor
video = VideoFileClip(video_file)
W, H = video.w, video.h

# Mostrar vídeo embebido usando IPython.display.Video
display(Video('video_karaoke.mp4', embed=True, width=W, height=H))
# Botón de descarga para el vídeo final
import ipywidgets as widgets
from google.colab import files

download_button = widgets.Button(
    description='Descargar video final',
    button_style='primary'
)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from google.colab import files
files.download("video_karaoke.mp4")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>